In [1]:
# import libraries (non-cryptogaphic)
import random # to generate phone numbers
import pandas as pd
from functools import reduce
import os

# import libraries (cryptographic)
import cryptography.hazmat.primitives.asymmetric.dh as dh
import hashlib
import sympy
import secrets

# Generate phone numbers and store them in phone_numbers.csv file

In [2]:
random.seed(10) # to ensure same phone numbers generated every time

In [3]:
# class to generate phone numbers for grab and gojek
class PhoneNumberGenerator:
    def __call__(self, count):
        phone_numbers = random.sample(range(80000000,100000000), count)
        return phone_numbers 

    
# class to store numbers to csv
class PhoneNumberStorageManager:
    def __init__(self):
        self.filename = "phone_numbers.csv"
    def __call__(self, gojek_phone_numbers, grab_phone_numbers):
        d = {"gojek": gojek_phone_numbers, 
            "grab": grab_phone_numbers}
        df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()])) # create dataframe
        df.to_csv(self.filename, index = False)

In [4]:
# generate and store phone numbers
gojek_phone_number_count = 61 # inclusive of phone numbers in common with grab
grab_phone_number_count = 91 # inclusive of phone numbers in common with gojek
common_phone_number_count = 10

# instantiate required classes
phone_number_generator = PhoneNumberGenerator()
phone_number_storage_manager = PhoneNumberStorageManager()

# generate phone numbers
phone_numbers = phone_number_generator(gojek_phone_number_count+grab_phone_number_count-common_phone_number_count)
common_phone_numbers = phone_numbers[0:common_phone_number_count]
gojek_phone_numbers = phone_numbers[0:gojek_phone_number_count]
grab_phone_numbers = common_phone_numbers + phone_numbers[gojek_phone_number_count:] 

#shuffle phone number lists
random.shuffle(gojek_phone_numbers)
random.shuffle(grab_phone_numbers)

# write phone numbers to csv file
phone_number_storage_manager(gojek_phone_numbers, grab_phone_numbers)

# Define classes for the necessary for the algorithm

In [5]:
# class to generate clients' private secret
class NumberGenerator:
    
    def generate_public_parameters(self, size):
        p = self.generate_safe_prime(size)
        print(f"p is prime: {sympy.ntheory.isprime(p)}")
        length_of_p = len(bin(p)[2:]) # should be 1024
        print(f"Length of prime modulus, p: {length_of_p}.\nNote: Should be {size}.")
        q = (p-1)//2
        print(f"q is prime: {sympy.ntheory.isprime(q)}")
        factors_pminus1 = [1, 2, q]
        
        return p, factors_pminus1

    def generate_safe_prime(self, size):
        candidate = dh.generate_parameters(2, size).parameter_numbers().p # generate 1024-bit prime number
        while True:
            is_safe_prime = sympy.ntheory.isprime((candidate-1)//2) # if safe prime, (candidate-1)/2 is prime
            if (is_safe_prime):
                break
            else:
                candidate = dh.generate_parameters(2, size).parameter_numbers().p
                print(candidate)
 
        return candidate
    
    def generate_random_number(self, size):
        return secrets.randbits(size)
            
# class to inspect values          
class NumberInspector:
    
    def check_is_primitive_generator(self, candidate, factors_divisorminus1, divisor): # note: factors should be the factors of divisor-1
        
        # apply lagrange theorem
        for possible_order in factors_divisorminus1: # check congruence for all factors (factors is exclusive of p-1 itself)
            result = pow(candidate, possible_order, divisor) # fast modular exponentiation
            if (result == 1):
                return False
            
        return True 

    
class StorageManager:
    
    def store_data(self, filename, data):
        df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe
        df.to_csv(filename, index=False)

                


    
        

# Create psi client class

In [6]:
# client class (both grab and gojek are clients communicating directly with each other)
class Client:
    def __init__(self, name, other_party_name, private_key_size, phone_numbers, p, factors_pminus1):
        
        self.number_inspector = NumberInspector()
        self.number_generator = NumberGenerator()
        
        self.private_key = self.number_generator.generate_random_number(private_key_size)
        self.my_set = phone_numbers
        self.p = p
        self.factors_pminus1 = factors_pminus1
        
        self.my_hashed_set = None
        self.my_self_encrypted_set = None
        self.my_encrypted_set = None
        self.other_party_encrypted_set = None
        self.common_values = None
        
        # create datafile for communication with another party
        # only need to share self_encrypted_values and other_party_encrypted_values
        self.my_dict = {
            'my_self_encrypted_set': None, 
            'other_party_encrypted_set': None,
            'common_values': None
        }
        
        self.name = name
        self.filename = name + "_data_v1.0.csv"
        self.other_party_name = other_party_name
        self.other_party_filename = other_party_name + "_data_v1.0.csv"
        
        self.storage_manager = StorageManager()
        self.storage_manager.store_data(self.filename, self.my_dict)

    def hash_to_primitive_root_modulo_p(self, element):     
        endian = "big"
        element = element.to_bytes(4, endian)
        hash_hex = hashlib.sha256(element).hexdigest() #sha3_256
        hash_int = int(hash_hex, 16)
        while True:
            is_primitive_generator = self.number_inspector.check_is_primitive_generator(
                hash_int, self.factors_pminus1, self.p
            )
            if (is_primitive_generator):
                break
            else:
                hash_int = hash_int.to_bytes(32, endian)
                hash_hex = hashlib.sha256(hash_int).hexdigest()
                hash_int = int(hash_hex, 16)
        return hash_int
    
    def modular_exponentation(self, element):
        return pow(element, self.private_key, self.p)
    
    def hash_set(self):
        
        self.my_hashed_set = []
        
        for element in self.my_set:
            hashed_value = self.hash_to_primitive_root_modulo_p(element)
            self.my_hashed_set.append(hashed_value)
            
    def encrypt_set(self, is_other_party):
        
        if (is_other_party):
            other_party_set = self.receive_data("my_self_encrypted_set")
            other_party_set_int = []
            for element_string in other_party_set:
                other_party_set_int.append(int(element_string))
            set_to_encrypt = other_party_set_int
        else:
            set_to_encrypt = self.my_hashed_set
        
        encrypted_values = []
        for element in set_to_encrypt:
            encrypted_value = self.modular_exponentation(element)
            encrypted_values.append(encrypted_value)
            
        if (is_other_party):
            self.other_party_encrypted_set = encrypted_values
            self.send_data(encrypted_values, "other_party_encrypted_set")
        else:
            self.my_self_encrypted_set = encrypted_values
            self.send_data(encrypted_values, "my_self_encrypted_set")
        
        
    def get_intersection(self):
        
        my_encrypted_set = self.receive_data("other_party_encrypted_set") 
        my_encrypted_set_int = []
        
        for element_string in my_encrypted_set:
            my_encrypted_set_int.append(int(element_string))
        self.my_encrypted_set = my_encrypted_set_int
        encrypted_common_values = set(self.my_encrypted_set).intersection(self.other_party_encrypted_set)
        index_of_common_values = []
        
        for element in encrypted_common_values:
            index_of_common_values.append(self.my_encrypted_set.index(element))
            
        self.common_values = []
        
        for index in index_of_common_values:
            self.common_values.append(self.my_set[index])
            
        self.my_dict["common_values"] = self.common_values
        self.send_data(self.common_values, "common_values")
    
    def send_data(self, data_to_send, column_name):
        # send data means writing to file
        self.my_dict[column_name] = data_to_send
        self.storage_manager.store_data(self.filename, self.my_dict)
        
        
    def receive_data(self, column_name):
        # receive data means reading from file
        data = self.get_other_party_data()[column_name].to_list()
        cleaned_data = [x for x in data if type(x) is not float]
        print(f"number of elements in {self.other_party_name};{column_name}: {len(cleaned_data)}.")
        return cleaned_data
        
    def get_my_data(self):
        return pd.read_csv(self.filename)
            
    def get_other_party_data(self):
        return pd.read_csv(self.other_party_filename)


    


# Initialize context

In [7]:
# assign pre-determined variables for psi
key_size = 1024 # both private keys and large prime

number_generator = NumberGenerator()
p, factors_pminus1 = number_generator.generate_public_parameters(key_size)

# create clients
grab = Client("grab", "gojek", key_size, grab_phone_numbers, p, factors_pminus1)
gojek = Client("gojek", "grab", key_size, gojek_phone_numbers, p, factors_pminus1)



p is prime: True
Length of prime modulus, p: 1024.
Note: Should be 1024.
q is prime: True


C:\Users\jiaji\AppData\Local\Temp/ipykernel_15152/525289407.py:47: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


# Get intersection

## Step 1: Hash phone numbers
Note: hashed set is not stored in the file as it is not meant to be shared with the other party hence, files' status after step 1 is not shown

In [8]:
# clients hash their own set
grab.hash_set()
gojek.hash_set()

### Status after step 1

In [9]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set
0,1061642916839729595311745988861392474934466737...,8093135284524341492510691949857707487260359145...
1,8697318885625964372170362596957150767783631535...,5930688136427591374346485570270456146366904370...
2,6456970480652929440281673297917938501061617606...,6500473151849576061038596173951167521581063508...
3,4271190063387899149690355220761209149002412790...,1722747940101829484199977620874807683223311608...
4,4320682999770379726688099447684191745405506617...,5750822890154339790428118545565634695520281701...
...,...,...
86,NaN,2625832542840555517519669698958112696515497351...
87,NaN,3683371775181761031500661295222456997304332645...
88,NaN,2408490635946548862425293650149893980450258320...
89,NaN,7019329188008094390450049845892045288943600434...


## Step 2: encrypt hashed set with own private key

In [10]:
# clients self encrypt hashed set
grab.encrypt_set(False) # set is_other_party to false to encrypt own hashed set
gojek.encrypt_set(False)

C:\Users\jiaji\AppData\Local\Temp/ipykernel_15152/525289407.py:47: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


### Clients' status after step 2 (value of variables in client)

In [11]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set
0,1061642916839729595311745988861392474934466737...,8093135284524341492510691949857707487260359145...,7544153208106280920519094621134236796877839762...,1780346139087002188990120491044645816116756944...
1,8697318885625964372170362596957150767783631535...,5930688136427591374346485570270456146366904370...,1149744806275630412313286319116095711276885526...,2121645270427910633297144015885566359904731804...
2,6456970480652929440281673297917938501061617606...,6500473151849576061038596173951167521581063508...,9408237547346902853837940879698220508991893562...,6383510711894679923945244137054744044313085600...
3,4271190063387899149690355220761209149002412790...,1722747940101829484199977620874807683223311608...,7693970747262260114450932750377880521036169989...,2778750601579254783559305932278861022957977962...
4,4320682999770379726688099447684191745405506617...,5750822890154339790428118545565634695520281701...,1037284057094040053569508541820699568565691954...,6958961223105889164980388461931248036995488068...
...,...,...,...,...
86,NaN,2625832542840555517519669698958112696515497351...,NaN,8158675962947687108528012291283486393725280338...
87,NaN,3683371775181761031500661295222456997304332645...,NaN,1469586685303093075165126586315311531373340262...
88,NaN,2408490635946548862425293650149893980450258320...,NaN,3705091017071389754450591994957621189588466206...
89,NaN,7019329188008094390450049845892045288943600434...,NaN,1167479363533203827304289924609821832529038641...


### Files' status after step 2 (value of variables in files)

In [12]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek

gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,7544153208106280920519094621134236796877839762...,NaN,NaN
1,1149744806275630412313286319116095711276885526...,NaN,NaN
2,9408237547346902853837940879698220508991893562...,NaN,NaN
3,7693970747262260114450932750377880521036169989...,NaN,NaN
4,1037284057094040053569508541820699568565691954...,NaN,NaN
...,...,...,...
56,1026416948256897373337419217373037369975650280...,NaN,NaN
57,4280423960795937690791500769099751159706555264...,NaN,NaN
58,1314355559917528473755294308311092765127619832...,NaN,NaN
59,1198285459529109664159600745998064815292436532...,NaN,NaN


In [13]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,1780346139087002188990120491044645816116756944...,NaN,NaN
1,2121645270427910633297144015885566359904731804...,NaN,NaN
2,6383510711894679923945244137054744044313085600...,NaN,NaN
3,2778750601579254783559305932278861022957977962...,NaN,NaN
4,6958961223105889164980388461931248036995488068...,NaN,NaN
...,...,...,...
86,8158675962947687108528012291283486393725280338...,NaN,NaN
87,1469586685303093075165126586315311531373340262...,NaN,NaN
88,3705091017071389754450591994957621189588466206...,NaN,NaN
89,1167479363533203827304289924609821832529038641...,NaN,NaN


## Step 3: encrypt other party's self-encrypted set with own private key

In [14]:
# clients encrypt other party's self encrypted set
grab.encrypt_set(True) # set is_other_party to true
gojek.encrypt_set(True)

number of elements in gojek;my_self_encrypted_set: 61.


C:\Users\jiaji\AppData\Local\Temp/ipykernel_15152/525289407.py:47: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


number of elements in grab;my_self_encrypted_set: 91.


### Clients' status after step 3

In [15]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set,
    "gojek encrypted set": grab.other_party_encrypted_set,
    "grab encrypted set": gojek.other_party_encrypted_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set,gojek encrypted set,grab encrypted set
0,1061642916839729595311745988861392474934466737...,8093135284524341492510691949857707487260359145...,7544153208106280920519094621134236796877839762...,1780346139087002188990120491044645816116756944...,6343447716978751604977591832609489333291333637...,1881355035924903210753270229699645658117056044...
1,8697318885625964372170362596957150767783631535...,5930688136427591374346485570270456146366904370...,1149744806275630412313286319116095711276885526...,2121645270427910633297144015885566359904731804...,7496062977374022902543784513061848899085015374...,4222979717193896842655260113189779856175631764...
2,6456970480652929440281673297917938501061617606...,6500473151849576061038596173951167521581063508...,9408237547346902853837940879698220508991893562...,6383510711894679923945244137054744044313085600...,7866330240891203540524036869803274230933574357...,1207882844079015350526254194424752093804870089...
3,4271190063387899149690355220761209149002412790...,1722747940101829484199977620874807683223311608...,7693970747262260114450932750377880521036169989...,2778750601579254783559305932278861022957977962...,7216267108912776022505273044227465483046132836...,5951803600081185909151412169137317576587600158...
4,4320682999770379726688099447684191745405506617...,5750822890154339790428118545565634695520281701...,1037284057094040053569508541820699568565691954...,6958961223105889164980388461931248036995488068...,7092564147127291375485080796687547525673327793...,1540360887385285280973731916988578210162750537...
...,...,...,...,...,...,...
86,NaN,2625832542840555517519669698958112696515497351...,NaN,8158675962947687108528012291283486393725280338...,NaN,1201316023332322871643049270548618106722989184...
87,NaN,3683371775181761031500661295222456997304332645...,NaN,1469586685303093075165126586315311531373340262...,NaN,9906380572134409778994073963340454650926689299...
88,NaN,2408490635946548862425293650149893980450258320...,NaN,3705091017071389754450591994957621189588466206...,NaN,4135699116592914961844287486526351830995635875...
89,NaN,7019329188008094390450049845892045288943600434...,NaN,1167479363533203827304289924609821832529038641...,NaN,4220932099625084160661817853597623978805498916...


### Files' status after step 3

In [16]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek

gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,7544153208106280920519094621134236796877839762...,1881355035924903210753270229699645658117056044...,NaN
1,1149744806275630412313286319116095711276885526...,4222979717193896842655260113189779856175631764...,NaN
2,9408237547346902853837940879698220508991893562...,1207882844079015350526254194424752093804870089...,NaN
3,7693970747262260114450932750377880521036169989...,5951803600081185909151412169137317576587600158...,NaN
4,1037284057094040053569508541820699568565691954...,1540360887385285280973731916988578210162750537...,NaN
...,...,...,...
86,NaN,1201316023332322871643049270548618106722989184...,NaN
87,NaN,9906380572134409778994073963340454650926689299...,NaN
88,NaN,4135699116592914961844287486526351830995635875...,NaN
89,NaN,4220932099625084160661817853597623978805498916...,NaN


In [17]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,1780346139087002188990120491044645816116756944...,6343447716978751604977591832609489333291333637...,NaN
1,2121645270427910633297144015885566359904731804...,7496062977374022902543784513061848899085015374...,NaN
2,6383510711894679923945244137054744044313085600...,7866330240891203540524036869803274230933574357...,NaN
3,2778750601579254783559305932278861022957977962...,7216267108912776022505273044227465483046132836...,NaN
4,6958961223105889164980388461931248036995488068...,7092564147127291375485080796687547525673327793...,NaN
...,...,...,...
86,8158675962947687108528012291283486393725280338...,NaN,NaN
87,1469586685303093075165126586315311531373340262...,NaN,NaN
88,3705091017071389754450591994957621189588466206...,NaN,NaN
89,1167479363533203827304289924609821832529038641...,NaN,NaN


## Step 4: find intersection

In [18]:
# clients find intersection
grab.get_intersection()
gojek.get_intersection()

number of elements in gojek;other_party_encrypted_set: 91.
number of elements in grab;other_party_encrypted_set: 61.


### Clients' status after step 4:

In [19]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set,
    "gojek encrypted set": grab.other_party_encrypted_set,
    "grab encrypted set": gojek.other_party_encrypted_set,
    "gojek found intersection": gojek.common_values,
    "grab found intersection:": grab.common_values}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set,gojek encrypted set,grab encrypted set,gojek found intersection,grab found intersection:
0,1061642916839729595311745988861392474934466737...,8093135284524341492510691949857707487260359145...,7544153208106280920519094621134236796877839762...,1780346139087002188990120491044645816116756944...,6343447716978751604977591832609489333291333637...,1881355035924903210753270229699645658117056044...,94391128.0,96192082.0
1,8697318885625964372170362596957150767783631535...,5930688136427591374346485570270456146366904370...,1149744806275630412313286319116095711276885526...,2121645270427910633297144015885566359904731804...,7496062977374022902543784513061848899085015374...,4222979717193896842655260113189779856175631764...,96192082.0,94391128.0
2,6456970480652929440281673297917938501061617606...,6500473151849576061038596173951167521581063508...,9408237547346902853837940879698220508991893562...,6383510711894679923945244137054744044313085600...,7866330240891203540524036869803274230933574357...,1207882844079015350526254194424752093804870089...,89312048.0,89312048.0
3,4271190063387899149690355220761209149002412790...,1722747940101829484199977620874807683223311608...,7693970747262260114450932750377880521036169989...,2778750601579254783559305932278861022957977962...,7216267108912776022505273044227465483046132836...,5951803600081185909151412169137317576587600158...,95521626.0,95521626.0
4,4320682999770379726688099447684191745405506617...,5750822890154339790428118545565634695520281701...,1037284057094040053569508541820699568565691954...,6958961223105889164980388461931248036995488068...,7092564147127291375485080796687547525673327793...,1540360887385285280973731916988578210162750537...,99173089.0,99173089.0
...,...,...,...,...,...,...,...,...
86,NaN,2625832542840555517519669698958112696515497351...,NaN,8158675962947687108528012291283486393725280338...,NaN,1201316023332322871643049270548618106722989184...,NaN,NaN
87,NaN,3683371775181761031500661295222456997304332645...,NaN,1469586685303093075165126586315311531373340262...,NaN,9906380572134409778994073963340454650926689299...,NaN,NaN
88,NaN,2408490635946548862425293650149893980450258320...,NaN,3705091017071389754450591994957621189588466206...,NaN,4135699116592914961844287486526351830995635875...,NaN,NaN
89,NaN,7019329188008094390450049845892045288943600434...,NaN,1167479363533203827304289924609821832529038641...,NaN,4220932099625084160661817853597623978805498916...,NaN,NaN


Note: Last 2 columns, unlike the rest of the columns, do not have a one-to-one mapping with other values belonging to the same row i.e. values in the last 2 columns do not have any relation to the other values in the same row as it.

### Files' status after step 4

In [20]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek

gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,7544153208106280920519094621134236796877839762...,1881355035924903210753270229699645658117056044...,94391128.0
1,1149744806275630412313286319116095711276885526...,4222979717193896842655260113189779856175631764...,96192082.0
2,9408237547346902853837940879698220508991893562...,1207882844079015350526254194424752093804870089...,89312048.0
3,7693970747262260114450932750377880521036169989...,5951803600081185909151412169137317576587600158...,95521626.0
4,1037284057094040053569508541820699568565691954...,1540360887385285280973731916988578210162750537...,99173089.0
...,...,...,...
86,NaN,1201316023332322871643049270548618106722989184...,NaN
87,NaN,9906380572134409778994073963340454650926689299...,NaN
88,NaN,4135699116592914961844287486526351830995635875...,NaN
89,NaN,4220932099625084160661817853597623978805498916...,NaN


In [21]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,1780346139087002188990120491044645816116756944...,6343447716978751604977591832609489333291333637...,96192082.0
1,2121645270427910633297144015885566359904731804...,7496062977374022902543784513061848899085015374...,94391128.0
2,6383510711894679923945244137054744044313085600...,7866330240891203540524036869803274230933574357...,89312048.0
3,2778750601579254783559305932278861022957977962...,7216267108912776022505273044227465483046132836...,95521626.0
4,6958961223105889164980388461931248036995488068...,7092564147127291375485080796687547525673327793...,99173089.0
...,...,...,...
86,8158675962947687108528012291283486393725280338...,NaN,NaN
87,1469586685303093075165126586315311531373340262...,NaN,NaN
88,3705091017071389754450591994957621189588466206...,NaN,NaN
89,1167479363533203827304289924609821832529038641...,NaN,NaN


# Check results

In [22]:
# get intersection found by the two parties
grab_data = grab.get_my_data()
gojek_data = gojek.get_my_data()
gojek_found_intersection = gojek_data["common_values"].tolist()
grab_found_intersection = grab_data["common_values"].tolist()

# sort numbers for easier comparison
gojek_found_intersection.sort()
grab_found_intersection.sort()
common_phone_numbers.sort()

# summarize them in a dataframe
d = {"actual": common_phone_numbers,
    "gojek": gojek_found_intersection,
    "grab": grab_found_intersection}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()])).dropna(how = 'all')
df

,actual,gojek,grab
0,80497694.0,80497694.0,80497694.0
1,81093373.0,81093373.0,81093373.0
2,86915509.0,86915509.0,86915509.0
3,89312048.0,89312048.0,89312048.0
4,94391128.0,94391128.0,94391128.0
5,95521626.0,95521626.0,95521626.0
6,96192082.0,96192082.0,96192082.0
7,96485172.0,96485172.0,96485172.0
8,99173089.0,99173089.0,99173089.0
9,99397525.0,99397525.0,99397525.0
